# Install Requirements

In [ ]:
# %pip install autosink-data-preparation
# %pip install autosink-data-validation
# %pip install autosink-model-evaluation
# %pip install autosink-model-training
# %pip install autosink-model-validation

In [ ]:
%pip install git+https://github.com/slink-to-unlock/autosink-data-preparation.git
%pip install git+https://github.com/slink-to-unlock/autosink-data-validation.git
%pip install git+https://github.com/slink-to-unlock/autosink-model-evaluation.git
%pip install git+https://github.com/slink-to-unlock/autosink-model-training.git
%pip install git+https://github.com/slink-to-unlock/autosink-model-validation.git
%pip install git+https://github.com/slink-to-unlock/sparse-to-dense.git

# Data Extraction

In [5]:
import os
from autosink_data_elt.path.autosink import AutosinkPath

path = AutosinkPath(data_lake_rel_dir=os.path.join('dev', 'autowash_dataset', 'data'))
data_dir = path.get_dataset_root_dir('v3')
data_dir

'/content/mnt/MyDrive/dev/autowash_dataset/data/v5'

In [ ]:
from google.colab import drive

drive.mount(path.mount_dir)

In [ ]:
from urllib import request
PORT = 8501

# run in background
!nohup python3 -m sparse_to_dense.cli.web --port {PORT} > logs.txt &
!nohup npx localtunnel --port {PORT} > logs-localtunnel.txt &

print("My IP (Password):", request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!sleep 5 && cat logs-localtunnel.txt && cat logs.txt

- 데이터 레이블링이 필요한 상황이라면 위 셀 실행 결과에 출력되는 링크를 여세요.
- 링크의 빈칸에 `My IP (Password)` 값을 복사해 넣으세요.
- `Autosink` 프로젝트의 레이블러를 Google COLAB 에서 실행시킬 수 있습니다.

# Data Preparation

# Data validation

# Model Training

# Model Evaluation

# Model Validation